# TTS API test

In [84]:

import pandas as pd
import requests
import json
import configparser as cp
import pydub 
import glob
import static_ffmpeg
static_ffmpeg.add_paths()

True

In [85]:
# !pip3 install pandas requests configparser pydub static-ffmpeg

#### Making dirs

In [86]:
if not os.path.exists('data/json'):
    os.makedirs('data/json')
    
if not os.path.exists('data/podcast_scripts'):
    os.makedirs('data/podcast_scripts')
    
if not os.path.exists('output/stems'):
    os.makedirs('output/stems')
    
if not os.path.exists('output/audio_combined'):
    os.makedirs('output/audio_combined')

In [87]:
# Getting api keys
config = cp.ConfigParser()
config.read('config.ini')
eleven_api_key = config.get('api_keys', 'elevenlabs')


## Getting speakers

In [88]:
url = "https://api.elevenlabs.io/v1/voices"

headers = {
    "accept": "application/json",
    "xi-api-key": eleven_api_key
}

response = requests.get(url, headers=headers)
print(response.status_code)

data = response.json()
with open("./data/json/speakers_eleven.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

200


## Reading in podcast script

In [94]:
# Defining podcast script to use
pod_script_fn = 'lavender_jiang.txt'

# Extracting basename
pod_script_fn_base = pod_script_fn.replace('.txt', '')

# Reading in script
with open('data/podcast_scripts/' + pod_script_fn, 'r') as file:
    pod_script = file.readlines()

# extracting host and guest dialogue
host_lines = [line for line in pod_script if line.startswith('**host:** ')]
host_lines = [i.replace("**host:** ", "") for i in host_lines]
host_lines = [i.strip() for i in host_lines]


guest_lines = [line for line in pod_script if line.startswith('**guest:** ')]
guest_lines = [i.replace("**guest:** ", "") for i in guest_lines]
guest_lines = [i.strip() for i in guest_lines]

In [90]:
host_lines

['Welcome to today\'s podcast! We have a special guest with us, Dr Lavender Jiang, the author of an exciting paper titled "Health system-scale language models are all-purpose prediction engines." The paper was published in Nature on June 7, 2023. Lavender, it\'s great to have you here. Can you give us a brief overview of your paper and its main findings?',
 'That sounds fascinating! Can you explain how NYUTron works and how it was trained?',
 "It's impressive how you've managed to use unstructured clinical notes to train such a powerful model. What were some of the tasks you evaluated NYUTron on, and how did it perform?",
 "It's amazing to see how versatile NYUTron is in handling various tasks. How do you envision NYUTron being integrated into clinical workflows and assisting healthcare professionals?",
 "I can see how this could be a game-changer in healthcare. However, I'm curious about the generalizability of NYUTron across different clinical environments. How does it perform in dif

In [91]:
guest_lines

['Thank you for having me! In our paper, we show that unstructured clinical notes from electronic health records can be used to train clinical language models, which can then be applied as all-purpose clinical predictive engines. We developed a large language model called NYUTron and fine-tuned it across a wide range of clinical and operational predictive tasks. Our results demonstrate the potential of using clinical language models in medicine to provide guidance at the point of care.',
 'Sure! NYUTron is based on a bidirectional encoder model known as BERT, and it was pretrained on a vast set of unlabelled clinical notes from the NYU Langone electronic health record system. We then fine-tuned the model for each downstream task using a masked language modeling objective, which involves masking words or subwords in clinical notes and training the model to fill in the masked word correctly.',
 'We evaluated NYUTron on five different tasks: 30-day all-cause readmission prediction, in-hos

## API functions

In [92]:
def text_to_voice_el(input_text, speaker, api_key, outdir, filename, stability = 0, similarity_boost = 0, style = 0, speaker_boost = True):

    url = 'https://api.elevenlabs.io/v1/text-to-speech/' + speaker

    headers = {
        "accept": "audio/mpeg",
        "Content-Type": "application/json",
        "xi-api-key": api_key
    }
    payload = {
    "text": input_text,
    "optimize_streaming_latency": 1,
    "model_id": "eleven_multilingual_v2",
    "voice_settings": {
            "stability": stability,
            "similarity_boost": similarity_boost,
            "style": style,
            "use_speaker_boost": speaker_boost
        }
    }

    response = requests.post(url, json=payload, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Open a new file in binary mode, 'wb'
        outpath = outdir + "/" + filename + '.mp3'
        print('success, writing .mp3 to ', outpath)
        with open(outpath, 'wb') as f:
            # Write the content of the response to the file
            f.write(response.content)
    else:
        print('Request failed with status code', response.status_code)
    
    

## Generating responses for host

In [96]:
# Generating responses for all lines
speaker_id = "ZQe5CZNOzWyzPSCn5a3c" # James

# Creating directory
if not os.path.exists('output/stems/' + pod_script_fn_base):
    os.makedirs('output/stems/' + pod_script_fn_base)

stems_output_dir = "output/stems/" + pod_script_fn_base

for i in range(0, len(host_lines)):
    input_text = host_lines[i]
    print("INPUT == ", input_text)
    text_to_voice_el(input_text, speaker_id, eleven_api_key, outdir=stems_output_dir, filename="host_" + str(i)) 


success, writing .mp3 to  output/stems/lavender_jiang/host_0.mp3
success, writing .mp3 to  output/stems/lavender_jiang/host_1.mp3
success, writing .mp3 to  output/stems/lavender_jiang/host_2.mp3
success, writing .mp3 to  output/stems/lavender_jiang/host_3.mp3
success, writing .mp3 to  output/stems/lavender_jiang/host_4.mp3
success, writing .mp3 to  output/stems/lavender_jiang/host_5.mp3


## Generating responses for guest

In [97]:
# Generating responses for all lines
speaker_id = "XrExE9yKIg1WjnnlVkGX" # kayla

for i in range(0, len(guest_lines)):
    input_text = guest_lines[i]
    print("INPUT == ", input_text)
    text_to_voice_el(input_text, speaker_id, eleven_api_key, outdir=stems_output_dir, filename="guest_" + str(i)) 


INPUT ==  Thank you for having me! In our paper, we show that unstructured clinical notes from electronic health records can be used to train clinical language models, which can then be applied as all-purpose clinical predictive engines. We developed a large language model called NYUTron and fine-tuned it across a wide range of clinical and operational predictive tasks. Our results demonstrate the potential of using clinical language models in medicine to provide guidance at the point of care.
success, writing .mp3 to  output/stems/lavender_jiang/guest_0.mp3
INPUT ==  Sure! NYUTron is based on a bidirectional encoder model known as BERT, and it was pretrained on a vast set of unlabelled clinical notes from the NYU Langone electronic health record system. We then fine-tuned the model for each downstream task using a masked language modeling objective, which involves masking words or subwords in clinical notes and training the model to fill in the masked word correctly.


# Combining mp3 files


In [ ]:
# listing host files 
host_files = glob.glob(stems_output_dir + 'host_*', )
host_files = sorted(host_files)

guest_files = glob.glob(stems_output_dir + 'guest_*')
guest_files = sorted(guest_files)

# Loading files into list
host_audio_list = [AudioSegment.from_file(i, format='mp3') for i in host_files]
guest_audio_list = [AudioSegment.from_file(i, format='mp3') for i in guest_files]


In [ ]:
# Interleaving and concatneating audio files 
combined_list = []
for i in zip(host_audio_list, guest_audio_list):
    combined_list.extend(i)

In [ ]:
# exporting

final_sound = AudioSegment.empty()

for i in combined_list:
    final_sound += i

# Creating directory
if not os.path.exists('output/audio_combined/' + pod_script_fn_base):
    os.makedirs('output/audio_combined/' + pod_script_fn_base)

audio_combined_outdir = 'output/audio_combined/' + pod_script_fn_base + '_output_audio.mp3'
final_sound.export(audio_combined_outdir, format = 'mp3')

<_io.BufferedRandom name='test.mp3'>